In [ ]:
import os, sys, gdal
%matplotlib inline
import matplotlib.pylab as plt
import matplotlib.patches as patches
from skimage import exposure
import numpy as np
import pandas as pd

In [ ]:
datadirectory = '../Retrieved Data/wa_v2/BIOsS1'
datefile = 'S32631X398020Y1315440sS1_A_vv_0001_mtfil.dates'
imagefile='S32631X398020Y1315440sS1_A_vv_0001_mtfil.vrt'
imagefile_cross = 'S32631X398020Y1315440sS1_A_vh_0001_mtfil.vrt'

In [ ]:
os.chdir(datadirectory)

In [ ]:
def CreateGeoTiff(Name, Array, DataType, NDV, bandnames = None, ref_image = None,
                 GeoT = None, Projection = None):
    # If it's a 2D image we fake a third dimension:
    if len(Array.shape)==2:
        Array = np.array([Array])
    if ref_image ==None and (GeoT == None or Projection == None):
        raise RuntimeWarning('ref_image or setting required.')
    if bandnames != None:
        if len(bandnames) != Array.shape[0]:
            raise RuntimeError('Need {} bandnames. {} given'
                               .format(Array.shape[0], len(bandnames)))
    else:
        bandnames = ['Band []'.format(i + 1) for i in range(Array.shape[0])]
    if ref_image != None:
        refimg = gdal.Open(ref_image)
        GeoT = refimg.GetGeoTransform()
        Projection = refimg.GetProjection()
    driver = gdal.GetDriverByName("GTIFF")
    Array[np.isnan(Array)] = NDV
    DataSet = driver.Create(Name,
                           Array.shape[2],Array.shape[1], Array.shape[0], DataType)
    DataSet.SetGeoTransform(GeoT)
    DataSet.SetProjection(Projection)
    for i, image in enumerate(Array, 1):
        DataSet.GetRasterBand(i).WriteArray(image)
        DataSet.GetRasterBand(i).SetNoDataValue(NDV)
        DataSet.SetDescription(bandnames[i-1])
    DataSet.FlushCache()
    return Name

In [ ]:
def timeseries_metrics(raster, ndv = 0):
    # Make us of numpy nan functions
    # Check if type is a float array
    if not raster.dtype.name.find('float')>-1:
        raster = raster.astype(np.flost32)
    # Set ndv to nan
    if ndv != np.nan:
        raster[np.equal(raster, ndv)] = np.nan
    # Build dictionary of the metircs
    tsmetrics = {}
    rperc = np.nanpercentile(raster, [5, 50, 95], axis = 0)
    tsmetrics['mean'] = np.nanmean(raster, axis = 0)
    tsmetrics['max'] = np.nanmax(raster, axis = 0)
    tsmetrics['min'] = np.nanmin(raster, axis = 0)
    tsmetrics['range'] = tsmetrics['max'] - tsmetrics['min']
    tsmetrics['median'] = rperc[1]
    tsmetrics['p5'] = rperc[0]
    tsmetrics['p95'] = rperc[2]
    tsmetrics['prange'] = rperc[2] - rperc[0]
    tsmetrics['var'] = np.nanvar(raster, axis = 0)
    tsmetrics['cov'] = tsmetrics['var']/tsmetrics['mean']
    return tsmetrics

In [ ]:
#  Get the date indices via pandas
dates = open(datefile).readlines()
tindex = pd.DatetimeIndex(dates)
j = 1
print('Bands and dates for', imagefile)
for i in tindex:
    print("{:4d} {}".format(j, i.date()), end = ' ')
    j += 1
    if j%5 == 1: print()

In [ ]:
img = gdal.Open(imagefile)
img.RasterCount, img.RasterYSize, img.RasterYSize

In [ ]:
subset = (1500, 0, 500, 500) # (xoff, yoff, xsize, ysize)
bandnbr = 1

rasterDN = img.GetRasterBand(bandnbr).ReadAsArray()
fig, ax = plt.subplots(figsize = (8,8))
ax.set_title('Sentinel-1 C-VV, NIGER!!!!!, {}'
            .format(tindex[bandnbr-1].date()))
ax.imshow(rasterDN, cmap = 'gray', vmin = 2000, vmax = 8000)
ax.grid(color = 'blue')
ax.set_xlabel('Pixels')
ax.set_ylabel('Lines')
# plot the subset as rectang;e
if subset != None:
    _ = ax.add_patch(patches.Rectangle((subset[0], subset[1]),
                                      subset[2], subset[3],
                                      fill = False, edgecolor = 'red',
                                      linewidth = 3))

In [ ]:
rasterDN = img.ReadAsArray(*subset)
mask = rasterDN == 0
CF = np.power(10.,-8.3)
rasterPwr = np.ma.array(np.power(rasterDN, 2.)*CF, mask = mask, dtype = np.float32)

In [ ]:
rgb_bands = (1, int(img.RasterCount/2), img.RasterCount) # first, center, last band
rgb_bands = (1, 10, 40)
rgb_bands = (18, 45, 74)
rgb_idx = np.array(rgb_bands) - 1 # gets array index from bands by subtracting 1
rgb = np.dstack((rasterPwr[rgb_idx[0]], rasterPwr[rgb_idx[1]], rasterPwr[rgb_idx[2]]))
rgb_dates = (tindex[rgb_idx[0]].date(),
            tindex[rgb_idx[1]].date(), tindex[rgb_idx[2]].date())

In [ ]:
rgb_stretched = rgb.copy()
# For each band we apply the stretch
for i in range(rgb_stretched.shape[2]):
    rgb_stretched[:,:,i] = exposure.\
    equalize_hist(rgb_stretched[:,:,i].data,
    mask = ~np.equal(rgb_stretched[:,:,i].data,0.))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (16, 8))
fig.suptitle('Multi-temporal Sentinel-1 backscatter image R:{} G:{} B:{}'
            .format(rgb_dates[0],rgb_dates[1],rgb_dates[2]))
plt.axis('off')
ax[0].imshow(rgb)
ax[0].set_title('Unstretched')
ax[0].axis('off')
ax[1].imshow(rgb_stretched)
ax[1].set_title('Histogram Equalized')
_ = ax[1].axis('off')

In [ ]:
metrics = timeseries_metrics(rasterPwr.filled(np.nan), ndv = np.nan)

In [ ]:
#  Print out what the vaious metrics keys are
metrics.keys()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (16, 4))
ax[0].hist(metrics['var'].flatten(), bins = 100)
ax[1].hist(metrics['cov'].flatten(), bins = 100)
_ = ax[0].set_title('Variance')
_ = ax[1].set_title('Coefficient of Variation')

In [ ]:
# List the metric keys you want to plot
metric_keys = ['mean', 'median', 'max', 'min', 'p95', 'p5', 'range', 'prange', 'var', 'cov']
fig = plt.figure(figsize = (16, 40))
idx = 1
for i in metric_keys:
    ax = fig.add_subplot(5, 2, idx)
    if i == 'var' : vmin , vmax = (0.0, 0.005)
    elif i == 'cov' : vmin, vmax = (0., 0.04)
    else:
        vmin, vmax = (0.0001, 0.3)
    ax.imshow(metrics[i], vmin = vmin, vmax = vmax, cmap = 'gray')
    ax.set_title(i.upper())
    ax.axis('off')
    idx +=1

In [ ]:
plt.hist(metrics['range'].flatten(), bins = 100, range = (0.,0.3))
_ = plt.axvline(0.27, color = 'red')

In [ ]:
thres = 0.25
plt.figure(figsize = (8,8))
mask = metrics['range']<thres # For display we prepare the inverse mask
maskpdiff = ~mask # Store this for later output
plt.imshow(mask, cmap = 'gray')
plt.legend(['$p_{95} - p_5 > 0.15$'], loc = 'center right')
_ = plt.title('Threshold Classifier on Percentile Difference ($P_{95} - P_5 > 0.15$)')

In [ ]:
plt.hist(metrics['cov'].flatten(), bins = 100, range = (0,0.05))
_ = plt.axvline(0.025, color = 'red')

In [ ]:
thres = 0.025
mask = metrics['cov'] < thres
maskcv = ~mask
plt.figure(figsize = (8,8))
plt.imshow(mask, cmap = 'gray')
_ = plt.title('Threshold Classifier on the Time Series Coefficient of Variation')

In [ ]:
tsmean = 10*np.log10(np.nanmean(rasterPwr.filled(np.nan), axis = (1, 2)))

In [ ]:
ts = pd.Series(tsmean, index = tindex)
for i in range(len(ts)):
    print(i, ts.index[i].date(), ts[i])

In [ ]:
Xr = rasterPwr[19]
Xi = rasterPwr[76]

In [ ]:
r = np.log10(Xi/Xr)

In [ ]:
#  Display r
fig, ax = plt.subplots(2, 1, figsize = (8, 16))
ax[0].axis('off')
ax[0].imshow(np.abs(r), vmin = 0, vmax = 0.3, cmap = 'gray')
_ = ax[1].hist(r.flatten(), bins = 100, range = (-0.4,0.4))

In [ ]:
stddev = np.std(r)
thres = 3* stddev
mask = np.logical_and(r>-1*thres, r<thres)
masklr = ~mask

In [ ]:
fig, ax = plt.subplots(figsize = (8, 16))
ax.imshow(mask, cmap = 'gray')
ax.xaxis.set_ticks([])
ax.yaxis.set_ticks([])
_ = ax.set_title('Log Ratio Classifies of the October 2016/2017 Log Ratio Images')

In [ ]:
proj = img.GetProjection()
geotrans = list(img.GetGeoTransform())

subset_xoff = geotrans[0] + subset[0] * geotrans[1]
subset_yoff = geotrans[3] + subset[1] * geotrans[5]
geotrans[0] = subset_xoff
geotrans[3] = subset_yoff
geotrans  = tuple(geotrans)
geotrans

In [ ]:
# Time Series Metrics as image:
# We make a new subdirectory where we will store the images
dirname = imagefile.replace('.vrt', '_tsmetrics2')
os.makedirs(dirname, exist_ok = True)
print(dirname)

In [ ]:
Names = [] #List to keep track of all the names
for i in metrics:
    # Name, Array, DataType, NDV, bandnames = None, ref_image
    Name = os.path.join(dirname, imagefile.replace('.vrt', '_' +i+ '.tif'))
    CreateGeoTiff(Name, metrics[i], gdal.GDT_Float32, np.nan, [i], GeoT = geotrans, Projection = proj)
    Names.append(Name)

In [ ]:
cmd = 'gdalbuildvrt -seperate -overwrite -vrtnodata nan '+\
    dirname+ '.vrt ' + ' '.join(Names)
# print(cmd)
os.system(cmd)

In [ ]:
os.getcwd()

In [ ]:
print('Time Series Metrics VRT Fole:\n',dirname+'.vrt')

In [ ]:
imagename = imagefile.replace('.vrt', '_thresholds.tif')
bandnames = ['Percentile', 'COV', 'Log Ratio']
Array = np.array([maskpdiff, maskcv, masklr])
CreateGeoTiff(imagename, Array, gdal.GDT_Byte, 0, bandnames, GeoT = geotrans, Projection = proj)